<a href="https://colab.research.google.com/github/adamserag1/Interpretability-for-VRDU-models/blob/main/finetuning/LayoutLM3-RVL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall torch torchvision torchaudio fastai -y
!pip uninstall datasets transformers evaluate captum seqeval -y

!pip install torch==2.6.0+cu124 torchvision==0.21.0+cu124 torchaudio==2.6.0+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install datasets==3.4.0 transformers==4.38.2 evaluate==0.4.0 seqeval==1.2.2 captum==0.8.0

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: datasets 3.4.0
Uninstalling datasets-3.4.0:
  Successfully uninstalled datasets-3.4.0
Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
Found existing installation: evaluate 0.4.0
Uninstalling evaluate-0.4.0:
  Successfully uninstalled evaluate-0.4.0
Found existing installation: captum 0.8.0
Uninstalling captum-0.8.0:
  Successfully uninstalled captum-0.8.0
Found existing installation: seqeval 1.2.2
Uninstalling seqeval-1.2.2:
  Successfully uninstalled seqeval-1.2.2
Looking in 

In [3]:
from PIL import Image, ImageDraw, ImageFont
import evaluate
from transformers import AutoProcessor, LayoutLMForSequenceClassification, set_seed
from datasets import load_dataset
import transformers as t
import torch
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


set_seed(0)


In [5]:
rvl_cdip = load_dataset("aharley/rvl_cdip")

README.md:   0%|          | 0.00/6.15k [00:00<?, ?B/s]

rvl_cdip.py:   0%|          | 0.00/4.80k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/2.64k [00:00<?, ?B/s]

The repository for aharley/rvl_cdip contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/aharley/rvl_cdip.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


rvl-cdip.tar.gz:   0%|          | 0.00/38.8G [00:00<?, ?B/s]

train.txt:   0%|          | 0.00/13.7M [00:00<?, ?B/s]

test.txt:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

val.txt:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/320000 [00:00<?, ? examples/s]

In [ ]:
model = LayoutLMv3ForSequeneceClassification.from_pretrained("microsoft/layoutlmv3-large")
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-large")

In [ ]:
train_labels = rvl_cdip["train"].features["names"]
idtolabel = {v: k for v, k in enumerate(train_labels)}
labeltoid = {k: v for v, k in enumerate(train_labels)}

In [ ]:
def encode(example):
  images = [Image.open(path).convert("RGB") for path in example["image_path"]]
  words = example["words"]
  boxes = example["bboxes"]
  labels = example["ner_tags"]
  encoding = processor(images,
                       words,
                       boxes=boxes,
                       word_labels=labels,
                       padding="max_length",
                       truncation=True,
                       return_tensors="pt")
  return encoding

In [ ]:
metric = evaluate.load("seqeval ")